In [129]:
import numpy as np
import pandas as pd

import os
import os.path as path
import wget


In [118]:
if not path.exists('Data'):
    os.mkdir('Data')

!rm Data/*

file = 'Data/Gaussiana.csv'
url = 'https://raw.githubusercontent.com/asegura4488/Database/main/MetodosComputacionalesReforma/Gaussiano.csv'
if not path.exists(file):
    Path_ = wget.download(url,file)
    print('File loaded')
else:
    Path_ = file

"rm" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [119]:
data = pd.read_csv(Path_)
x = data.x

p= np.array([np.mean(x),np.std(x)])

In [120]:
def Prior(p):
    
    mu, sigma = p
    
    if 3 <= mu <= 5. and 0.5 <= sigma <= 3.5:
        return 1
    else:
        return 0

In [121]:
def Gauss(p,x):
    
    mu, sigma = p
    
    return np.exp( -0.5*(x-mu)**2/sigma**2  )/np.sqrt(2*np.pi*sigma**2)

In [122]:
def JointLikelihood(p,x):
    return np.sum( np.log(Gauss(p,x)) )

In [123]:
def LogPosterior(p,x):
    
    LogP = Prior(p)
    
    if not np.isfinite(LogP):
        return -np.inf
    else:
        return JointLikelihood(p,x) + LogP

In [124]:
LogPosterior(p,x)

-199.9809161045714

In [125]:
def Metropolis(N,p,x, likelihood_func):
    
    samples = [p]
    
    for i in range(N):
        
        p_new = np.random.normal()  # Generar una propuesta de un nuevo parámetro p   
        if 0 < p_new < 1:
            
            likelihood_new = likelihood_func(x,p_new)
            likelihood_current = likelihood_func(x,p)
            acceptance_ratio = likelihood_new / likelihood_current

            if np.random.uniform(0, 1) < acceptance_ratio.any() >= 1:
                
                p = p_new
        
        samples.append(p)
    print("Parametros=" , np.mean(samples), np.std(samples))
    return samples

In [131]:
Metropolis(20000,x,p,Gauss)

TypeError: loop of ufunc does not support argument 0 of type Series which has no callable conjugate method

In [ ]:
def calculate_interval(samples, confidence=0.69):
    
    lower_percentile = (1 - confidence) / 2
    upper_percentile = 1 - lower_percentile

    lower_bound = np.percentile(samples, lower_percentile * 100)
    upper_bound = np.percentile(samples, upper_percentile * 100)

    return lower_bound, upper_bound

In [ ]:
a=Metropolis(20000,x,p,Gauss)
calculate_interval(a)